In [ ]:
import time, torch
# diffusers imports are weird, helper:
# from diffusers import
from diffusers.utils.export_utils import export_to_video
from diffusers.pipelines.pipeline_utils import DiffusionPipeline
from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
from diffusers.pipelines.text_to_video_synthesis.pipeline_text_to_video_synth_img2img import VideoToVideoSDPipeline
from PIL import Image

pos_prompt = open("input-pos.txt", "r").read()
neg_prompt = open("input-neg.txt", "r").read()
cfg = 7

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
assert isinstance(pipe, VideoToVideoSDPipeline)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

video_frames = pipe(pos_prompt, negative_prompt=neg_prompt, num_inference_steps=25, guidance_scale=cfg).frames # type: ignore
fname = str(time.time()) + ".mp4"
video_path = export_to_video(video_frames, fname) # type: ignore
video_path

In [ ]:
# upscale
pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_XL", torch_dtype=torch.float16)
assert isinstance(pipe, VideoToVideoSDPipeline)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# memory optimization
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
pipe.enable_vae_slicing()

video = [Image.fromarray(frame).resize((1024, 576)) for frame in video_frames]

video_frames = pipe(pos_prompt, negative_prompt=neg_prompt, guidance_scale=cfg, video=video, strength=0.6).frames # type: ignore
fname = fname.replace(".mp4", "_upscale.mp4")
video_path = export_to_video(video_frames, fname) # type: ignore
video_path